In [ ]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
import ngsolve
from xfem import *

In [ ]:
maxh = 0.2
# Finite element space order
order = 5
# Stabilization parameter for ghost-penalty
gamma_stab = 100
# Stabilization parameter for Nitsche
lambda_nitsche = 10 * order * order

r = sqrt(x**2 + y**2)
levelset = r-1

beta0 =1
beta1 = 1
beta2 = 1
beta3 = 1
square = SplineGeometry()
square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
ngmesh = square.GenerateMesh(maxh=maxh)
mesh = Mesh(ngmesh)
Draw(mesh)

In [ ]:
lsetp1 = GridFunction(H1(mesh))
InterpolateToP1(levelset,lsetp1)# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
neg = ci.GetElementsOfType(NEG)
hasif = ci.GetElementsOfType(IF)
haspos = ci.GetElementsOfType(HASPOS)
ba_facets = GetFacetsWithNeighborTypes(mesh, a=haspos, b=any)
patch_elements = ci.GetElementsOfType(ANY)
#cut_elements = ci.GetElementsOfType(HASIF)  


interior_facets = GetFacetsWithNeighborTypes(mesh, a=neg, b=neg)
interface_facet_set = GetFacetsWithNeighborTypes(mesh, a=hasif, b=hasneg)

h = specialcf.mesh_size
n = Normalize(grad(lsetp1))

# integration domains:
dx = dCut(lsetp1, NEG, definedonelements=hasneg)
ds = dCut(lsetp1, IF, definedonelements=hasif)

dw = dFacetPatch(definedonelements=ba_facets)
dw_int = dFacetPatch(definedonelements=interior_facets)
dw_interface = dFacetPatch(definedonelements=interface_facet_set)


In [ ]:
V = VectorH1(mesh, order=order,dgjumps=True)
V = Compress(V, GetDofsOfElements(V,hasneg))
Q = H1(mesh, order=order-1)
Q = Compress(Q, GetDofsOfElements(Q,hasneg))
Z = NumberSpace(mesh)
X = V*Q*Z
(u,p,z),(v,q,y) = X.TnT()
gfu = GridFunction(X)

def jump(f):
    return f - f.Other()
def jumpn(f):
    n = Normalize(grad(lsetp1))
    return Grad(f)*n - Grad(f).Other()*n

a = BilinearForm(X)
stokes = InnerProduct(Grad(u), Grad(v))*dx + div(u)*q*dx + div(v)*p*dx
a += stokes
a += -(Grad(u)*n * v + Grad(v)*n * u) * ds + gamma_stab / h * u * v * ds #nitshe stabilization for boundary conditions
a += -(q*n * u + p*n * v) * ds
a += p*y *dx + q *z*dx
a += beta2* InnerProduct(jump(Grad(u)), jump(Grad(v))) * dw_interface #velocity ghost penalty
a += -beta0 * InnerProduct(jump(p), jump(q)) * dw_interface #velocity ghost penalty
#a += beta2/h* InnerProduct(Grad(u), Grad(v)) * ds #velocity ghost penalty
#a += SymbolicFacetBFI(beta2 / h * InnerProduct(jump(grad(u)), jump(grad(v)))*dx,element_boundary=True,definedon=interface_facet_set)
#a_vel = BilinearForm(V)
#a_vel += beta2 / h * InnerProduct(Grad(u), Grad(v)) * dw_interface
#a_vel.Assemble()

# Dann separat zur globalen Matrix addieren:
a.Assemble()
#a.mat += a_vel.mat

f = CF((0,x))
rhs = LinearForm(X)  # oder f*v*dx mit f gegeben
rhs += InnerProduct(f, v)* dx
rhs.Assemble()

gfu.vec.data = a.mat.Inverse(X.FreeDofs()) * rhs.vec

#l2error.append(sqrt(Integrate((gfu - exact)**2*dx, mesh)))
#Draw(gfu.components[0], mesh, "u")
#Draw(gfu.components[1], mesh, "p")

DrawDC(lsetp1,gfu.components[0], CF((0,0)), mesh, "u")
DrawDC(lsetp1,gfu.components[1], 0, mesh, "p", min = -0.1, max = 0.1, autoscale=False)
